In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [11]:
ib.disconnect()

In [2]:
from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *

In [17]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

history = '7 D'
freq = '1 min'
side ='Trades'

In [21]:
# Pull data from ADRs in "Asian_ADRs.csv"
adr_symbol = "ATHE"
adr_exchange = 'SMART'
adr_currency = "USD"

underlying_symbol = 'ATH'
underlying_exchange = 'SMART'
underlying_currency = 'AUD'
underlying_country = "Australia"

folder_name = underlying_country + "/" + adr_symbol + "_" + underlying_symbol + "/"

In [22]:
adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
ib.qualifyContracts(adr_contract)
adr_df = get_data(adr_contract, history, freq, side)
print(adr_df.head(3))
adr_df.to_csv(folder_name + "adr_week_min.csv")

                 date  open  high   low  close  volume  average  barCount
0 2021-04-08 09:30:00  1.42  1.42  1.42   1.42       2     1.42         1
1 2021-04-08 09:31:00  1.42  1.42  1.42   1.42       0     1.42         0
2 2021-04-08 09:32:00  1.42  1.42  1.42   1.42       0     1.42         0


In [23]:
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)
underlying_df = get_data(underlying_contract, history, freq, side)
print(underlying_df.head(3))
underlying_df.to_csv(folder_name + "underlying_week_min.csv")

                 date   open   high    low  close  volume  average  barCount
0 2021-04-07 19:59:00  0.032  0.032  0.032  0.032   20000    0.032         3
1 2021-04-07 20:00:00  0.032  0.032  0.032  0.032       0    0.032         0
2 2021-04-07 20:01:00  0.032  0.032  0.032  0.032       0    0.032         0


In [ ]:
ADR_Underlying = []

for pair in ADR_Underlying:
    # Pull data from ADRs in "Asian_ADRs.csv"
    adr_symbol = "GENE"
    adr_exchange = 'SMART'
    adr_currency = "USD"

    underlying_symbol = 'GTG'
    underlying_exchange = 'SMART'
    underlying_currency = 'AUD'
    underlying_country = "Australia"

    folder_name = underlying_country + "/" + adr_symbol + "_" + underlying_symbol + "/"
    
    adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
    ib.qualifyContracts(adr_contract)
    adr_df = get_data(adr_contract, history, freq, side)
    print(adr_df.head(3))
    adr_df.to_csv(folder_name + "adr_week_min.csv")
    
    underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
    ib.qualifyContracts(underlying_contract)
    underlying_df = get_data(underlying_contract, history, freq, side)
    print(underlying_df.head(3))
    underlying_df.to_csv(folder_name + "underlying_week_min.csv")

In [6]:
def AUS_opening(date_now):
#     aus_dst = {
#         2021: (date(2021, 4, 4), date(2021, 10, 3)),
#         2020: (date(2020, 4, 5), date(2020, 10, 4)),
#         2019: (date(2019, 4, 7), date(2019, 10, 6)),
#         2018: (date(2018, 4, 1), date(2018, 10, 7)),
#         2017: (date(2017, 4, 2), date(2017, 10, 1)),
#         2016: (date(2016, 4, 3), date(2016, 10, 2)),
#         2015: (date(2015, 4, 5), date(2015, 10, 4)),
#     }
    
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    
    us_start, us_end = us_dst[date_now.year]
    if us_start < date_now and date_now < us_end:
        return datetime.combine(date_now,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_now, time(10,0)) - timedelta(hours = 15)

def HK_opening(date_now):
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_now, time(9,30)) - timedelta(hours = 13)
    
def JPN_opening(date_now):
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)
    

In [6]:
forex_pairs = [["AUD",  "USD", "USD"],["USD", "HKD", "JPY",]]
opening_hours = [AUS_opening, HK_opening, JPN_opening]
df_dict = {}
forex_contracts = []
day_block = 5
history = str(day_block) + ' D'
freq = '1 min'
side ='BID_ASK'

In [ ]:
# forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
# ib.qualifyContracts(forex_contract)
# forex_df = get_data(forex_contract, history, freq, side, enddate)
# print(forex_df.head(3))
# forex_df.to_csv(folder_name + currency1 + "_" + currency2 + "_test.csv")
# df_dict[currency1+"."+currency2].concat(forex_df)

In [9]:
date_pointer = datetime(2020,5,21)
end_date = datetime.now()

while date_pointer < end_date:    
    debug_start_time = datetime.now()
    
    start_date = date_pointer
    close_date = min(date_pointer + timedelta(days = 28),end_date)

    # Initialize Dictionary to store dataframes and initialize all the contracts
    df_dict = {}
    forex_contracts = []
    
    for i in range(3):
        currency1 = forex_pairs[0][i]
        secType = "CASH"
        currency2 = forex_pairs[1][i]
        exchange = "IDEALPRO"

        df_dict[currency1+"."+currency2] = []

        forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
        ib.qualifyContracts(forex_contract)
        forex_contracts.append(forex_contract)

    # Iterate over the datesace
    while date_pointer < close_date:
        print(date_pointer.date())

#         if date_pointer.weekday() >5: 
#             date_pointer += timedelta(days = 1)
#             continue

        for i in range(3):
            currency1 = forex_pairs[0][i]
            currency2 = forex_pairs[1][i]
            contract = forex_contracts[i]

            # NYSE Opening 
            enddate = date_pointer
            forex_df_1 = get_data(contract, history, freq, side, enddate)
            df_dict[currency1+"."+currency2].append(forex_df_1)
            ib.sleep(1)

#             # Other Exchange Opening
#             enddate = opening_hours[i](date_pointer) + timedelta(minutes = 2)
#             forex_df_2 = get_data(contract, history, freq, side, enddate)
#             df_dict[currency1+"."+currency2].append(forex_df_2)

        date_pointer += timedelta(days = day_block)
        

    folder_name = "Forex/"
    for i in range(3):
        currency1 = forex_pairs[0][i]
        currency2 = forex_pairs[1][i]
        df_forex = pd.concat(df_dict[currency1+"."+currency2], ignore_index = True,verify_integrity = True).sort_values(by = "date")
        df_forex.to_csv(folder_name + currency1 + "_" + currency2 + "_minute_"+start_date.strftime("%Y%m%d")+"-"
                        +close_date.strftime("%Y%m%d")+".csv")

    debug_end_time = datetime.now()
    time_taken = debug_end_time - debug_start_time
    print(time_taken)

2020-05-21


NameError: name 'day_block' is not defined

Without sleep 0:06:53.159906

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
ax.plot(, y)
ax.set_title('A single plot')

This approach is flawed

In [7]:
forex_pairs = [["AUD",  "USD", "USD"],["USD", "HKD", "JPY",]]
opening_hours = [AUS_opening, HK_opening, JPN_opening]
df_dict = {}
forex_contracts = []
history = '300 S'
freq = '1 min'
side ='BID_ASK'

In [10]:
date_pointer = datetime(2020,5,21,9,30)
end_date = datetime.now()

while date_pointer < end_date:    
    debug_start_time = datetime.now()
    
    start_date = date_pointer
    close_date = min(date_pointer + timedelta(days = 30),end_date)

    # Initialize Dictionary to store dataframes and initialize all the contracts
    df_dict = {}
    forex_contracts = []
    
    for i in range(3):
        currency1 = forex_pairs[0][i]
        secType = "CASH"
        currency2 = forex_pairs[1][i]
        exchange = "IDEALPRO"

        df_dict[currency1+"."+currency2] = []

        forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
        ib.qualifyContracts(forex_contract)
        forex_contracts.append(forex_contract)

    # Iterate over the datesace
    while date_pointer < close_date:
        print(date_pointer.date())

        if date_pointer.weekday() >5: 
            date_pointer += timedelta(days = 1)
            continue

        for i in range(3):
            currency1 = forex_pairs[0][i]
            currency2 = forex_pairs[1][i]
            contract = forex_contracts[i]

            # NYSE Opening 
            enddate = date_pointer + timedelta(minutes = 2)
            forex_df_1 = get_data(contract, history, freq, side, enddate)
            df_dict[currency1+"."+currency2].append(forex_df_1)
            ib.sleep(1)

            # Other Exchange Opening
            enddate = opening_hours[i](date_pointer) + timedelta(minutes = 2)
            forex_df_2 = get_data(contract, history, freq, side, enddate)
            df_dict[currency1+"."+currency2].append(forex_df_2)
            ib.sleep(1)

        date_pointer += timedelta(days = 1)
        

    folder_name = "Forex/"
    for i in range(3):
        currency1 = forex_pairs[0][i]
        currency2 = forex_pairs[1][i]
        df_forex = pd.concat(df_dict[currency1+"."+currency2], ignore_index = True,verify_integrity = True).sort_values(by = "date")
        df_forex.to_csv(folder_name + currency1 + "_" + currency2 + "_minute_"+start_date.strftime("%Y%m%d")+"-"
                        +close_date.strftime("%Y%m%d")+".csv")

    debug_end_time = datetime.now()
    time_taken = debug_end_time - debug_start_time
    print(time_taken)

2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-13
2020-06-14
2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
0:11:55.043870
2020-06-20
2020-06-21
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-19
0:11:59.979623
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-26
2020-07-27
2020-07-28
2020-07-29
2020-07-30
2020-07-31
2020-08-01
2020-08-02
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-08
2020-08-09
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-15
2020-08-16
20

Peer closed connection


In [21]:
df_forex

,date,open,high,low,close,volume,average,barCount
5,2021-03-14 19:57:00,109.119,109.122,109.117,109.121,-1,-1.0,-1
6,2021-03-14 19:58:00,109.117,109.125,109.110,109.119,-1,-1.0,-1
7,2021-03-14 19:59:00,109.113,109.120,109.110,109.115,-1,-1.0,-1
8,2021-03-14 20:00:00,109.119,109.127,109.114,109.120,-1,-1.0,-1
9,2021-03-14 20:01:00,109.088,109.117,109.074,109.090,-1,-1.0,-1
0,2021-03-14 23:57:00,109.130,109.137,109.124,109.131,-1,-1.0,-1
1,2021-03-14 23:58:00,109.128,109.133,109.125,109.129,-1,-1.0,-1
2,2021-03-14 23:59:00,109.122,109.127,109.119,109.123,-1,-1.0,-1
3,2021-03-15 00:00:00,109.125,109.128,109.124,109.127,-1,-1.0,-1
4,2021-03-15 00:01:00,109.131,109.134,109.125,109.132,-1,-1.0,-1


In [7]:
df_list = []
folder_name = "Forex/"
for i in range(3):
    currency1 = forex_pairs[0][i]
    currency2 = forex_pairs[1][i]
    df_forex = pd.concat(df_dict[currency1+"."+currency2], ignore_index = True,verify_integrity = True).sort_values(by = "date")
    df_forex.to_csv(folder_name + currency1 + "_" + currency2 + "_minute.csv")
    df_list.append(df_forex)
    print(df_forex.head(10))

                 date     open     high      low    close  volume  average  \
5 2021-03-15 19:57:00  0.77482  0.77485  0.77478  0.77484      -1     -1.0   
6 2021-03-15 19:58:00  0.77475  0.77485  0.77469  0.77478      -1     -1.0   
7 2021-03-15 19:59:00  0.77472  0.77482  0.77470  0.77475      -1     -1.0   
8 2021-03-15 20:00:00  0.77491  0.77497  0.77478  0.77492      -1     -1.0   
9 2021-03-15 20:01:00  0.77480  0.77495  0.77469  0.77482      -1     -1.0   
0 2021-03-16 09:27:00  0.77517  0.77525  0.77512  0.77518      -1     -1.0   
1 2021-03-16 09:28:00  0.77493  0.77518  0.77470  0.77495      -1     -1.0   
2 2021-03-16 09:29:00  0.77472  0.77492  0.77462  0.77474      -1     -1.0   
3 2021-03-16 09:30:00  0.77479  0.77499  0.77460  0.77481      -1     -1.0   
4 2021-03-16 09:31:00  0.77483  0.77505  0.77463  0.77484      -1     -1.0   

   barCount  
5        -1  
6        -1  
7        -1  
8        -1  
9        -1  
0        -1  
1        -1  
2        -1  
3        -1  
4

In [13]:
history = '1 y'
freq = '30 mins'
side ='BID_ASK'

currency1 = forex_pairs[0][0]
secType = "CASH"
currency2 = forex_pairs[1][0]
exchange = "IDEALPRO"

enddate = datetime(2016, 4, 15)

folder_name = "Forex/"

In [14]:
forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
ib.qualifyContracts(forex_contract)
forex_df = get_data(forex_contract, history, freq, side, enddate)
print(forex_df.head(3))
forex_df.to_csv(folder_name + currency1 + "_" + currency2 + "_test.csv")
df_dict[currency1+"."+currency2].concat(forex_df)

reqHistoricalData: Timeout for Contract(secType='CASH', conId=14433401, symbol='AUD', exchange='IDEALPRO', currency='USD', localSymbol='AUD.USD', tradingClass='AUD.USD')


AttributeError: 'NoneType' object has no attribute 'head'

Error 162, reqId 179: Historical Market Data Service error message:API historical data query cancelled: 179, contract: Contract(secType='CASH', conId=14433401, symbol='AUD', exchange='IDEALPRO', currency='USD', localSymbol='AUD.USD', tradingClass='AUD.USD')


In [82]:
test = df_dict["AUD.USD"]

In [90]:
test[[timestamp.time() == time(17,16) for timestamp in test["date"]]]

,date,open,high,low,close,volume,average,barCount
1,2015-12-30 17:16:00,0.72837,0.72848,0.72829,0.72844,-1,-1.0,-1


In [78]:
time(17,16).tzinfo